In [1]:
# These are the python libraries we will use. 
# pandas is our python version of excel
# requests_html is our python version of a web browser (like chrome, or edge, or firefox) that we're going to 
# use to open the websites we want to get data from
import pandas as pd

In [2]:
url = 'https://fantasy.premierleague.com/statistics'

In [3]:
from selenium import webdriver
driver = webdriver.Chrome()

In [4]:
driver.get(url)

In [5]:
el = driver.find_element_by_id('sort')

In [6]:
for e in driver.find_elements_by_tag_name('button'):
    if e.text == 'Next':
        next_button = e
    if e.text == 'Previous':
        previous_button = e

In [10]:
next_button.click()

In [30]:
el.find_elements_by_tag_name('option')[2].click()

In [ ]:
data_tables = []
for option in el.find_elements_by_tag_name('option'):
    print(option.text)
    option.click()
    
    data_table = []
    
    for i in range(18-1):
        table = driver.find_element_by_tag_name('table')
        partial_data_table = pd.read_html(table.get_attribute('outerHTML'))[0]
        data_table.append(partial_data_table)
        next_button.click()

    data_table = pd.concat(data_table)
    data_table.set_index('Player', inplace=True)
    data_table.columns = [option.text if c == '**' else c for c in data_table.columns ]
    data_tables.append(data_table)
    

Total points
Round points
Price


In [49]:
combined_table = pd.concat(data_tables, axis=1)

In [50]:
combined_table.sort_index(axis='columns', inplace=True)

In [59]:
parsed_names = combined_table.index.to_series().apply(lambda x: pd.Series([x[:-6], x[-6:-3], x[-3:]], index=['Player Name', 'Team', 'Position']))

In [51]:
combined_table = combined_table.loc[:,~combined_table.columns.duplicated()]

In [62]:
combined_table = pd.concat([parsed_names, combined_table], axis=1)

In [63]:
combined_table.to_csv('Fantasy_PL_data.csv')